## Granular example

In [1]:
from pyo import *
s = Server(audio='jack').boot()

In [2]:
s.start()
snd = SndTable("../wavs/Reich_Music18.wav")
env = HannTable()
pos = Phasor(freq=snd.getRate()*.25, mul=snd.getSize())
dur = Noise(mul=.001, add=.1)
g = Granulator(snd, env, [1, 1.001], pos, dur, 32, mul=.1).out()

In [ ]:
s.gui(locals())

In [3]:
s.stop()

## Midi controller

In [1]:
from pyo import *

In [2]:
names, indexes = pm_get_input_devices()
print(names,indexes)

(['Midi Through Port-0', 'SL MkII MIDI 1', 'SL MkII MIDI 2', 'Pure Data Midi-Out 1'], [1, 3, 5, 8])


In [3]:
s = Server(audio='jack')

In [4]:
s.setMidiInputDevice(3) # Must be called before s.boot()
s.boot()
s.start()

In [14]:
s.stop()
s.start()
snd = SndTable("../wavs/Teen_Town.wav")
env = HannTable()

noin = Notein(poly=1,scale=0)

posx = Port( Midictl(ctlnumber=[78], minscale=0, maxscale=snd.getSize()), 0.02)
posf = Port( Midictl(ctlnumber=[16], minscale=0, maxscale=snd.getSize()), 0.02)
porta = Midictl(ctlnumber=[79], minscale=0., maxscale=60.)


posxx = (noin['pitch']-48.)/(96.-48.)*posf+posx
pos = SigTo(posxx)

def set_ramp_time():
    pos.time = porta.get()

tf = TrigFunc(Change(porta), function=set_ramp_time)

pitch = Port(Midictl(ctlnumber=[17], minscale=0.0, maxscale=2.0),0.02)

noisemul = Midictl(ctlnumber=[18], minscale=0.0, maxscale=0.2)

noiseadd = Port(Midictl(ctlnumber=[19], minscale=0.0, maxscale=1.0),0.02)

dur = Noise(mul=noisemul)+noiseadd

g = Granulator(snd, env, pitch*0.1/dur, pos , dur, 16, mul=.3).mix(2).out()

In [15]:
s.stop()

In [ ]:
s.gui(locals())

## Magnitud smoothing

In [1]:
from pyo import *
s = Server()

In [2]:
s.boot()

In [3]:
snd = "../wavs/Reich_Music18.wav"
size, olaps = 1024, 4
snd = SfPlayer(snd, speed=1.0, loop=True, mul=1.0)
fin = FFT(snd, size=size, overlaps=olaps)
pol = CarToPol(fin["real"], fin["imag"])
vec = Vectral(pol["mag"], framesize=size, overlaps=olaps, up=0.2, down=.01, damp=.01)
car = PolToCar(vec, pol["ang"])
fout = IFFT(car["real"], car["imag"], size=size, overlaps=olaps).mix(2).out()
s.start()

In [ ]:
s.stop()
s.shutdown()

## PV Additive Synth

In [2]:
from pyo import *
s = Server(audio='jack').boot()

In [9]:
s.start()
a = SfPlayer("../wavs/Reich_Music18.wav", loop=True, mul=0.7)
pva = PVAnal(a, size=1024, overlaps=4, wintype=2)
pvs = PVAddSynth(pva, pitch=1., num=500, first=10, inc=10).mix(2).out()

In [10]:
s.stop()

## Spectrum vis

In [ ]:
from pyo import *
s = Server().boot()
s.start()
a = SuperSaw(freq=[500,750], detune=0.6, bal=0.7, mul=0.5).out()
spec = Spectrum(a, size=1024)

In [ ]:
s.gui(locals())